# Notebook n. 1:

This is a first walk-through some of the functionalities of pyRTX. 
In this example we will show how to set up the code to compute the
Solar Radiation Pressure (SRP) considering the 3D shape of the spacecraft
and different levels of complexity for the sun photons modeling. 

Details about the spacrcraft shape modeling are treated in a different notebook

In [1]:
# Imports
import spiceypy as sp
import xarray as xr
import matplotlib.pyplot as plt
import logging, timeit

from pyRTX.classes.Spacecraft import Spacecraft
from pyRTX.classes.Planet import Planet
from pyRTX.classes.PixelPlane import PixelPlane
from pyRTX.classes.RayTracer import RayTracer
from pyRTX.classes.SRP import SunShadow, SolarPressure 
from pyRTX.classes.Precompute import Precompute
from pyRTX.core.analysis_utils import epochRange2
from pyRTX.visual.utils import plot_mesh
import logging

from numpy import floor, mod

import warnings
warnings.filterwarnings('ignore')

# Managing Error messages from trimesh
# (when concatenating textures, in this case, withouth .mtl definition, trimesh returns a warning that
#  would fill the stdout. Deactivate it for a clean output)
log = logging.getLogger('trimesh')
log.disabled = True

ImportError: Embree is not available. This feature requires Embree.
Please install it by running: python install_deps.py
On Linux, the environment should be configured automatically.
On other platforms, you may need to manually source embree-vars.sh

In [ ]:
# For all calculations requiring geometry/astrodynamic data we use SPICE
# Thus a metakernel containing attitude/trajectory information must be provided
# See the one provided in the example_data for an example
METAKR = '../example_data/LRO/metakernel_lro.tm'     # metakernel
sp.furnsh(METAKR)  # Load the metakernel containing references to the necessary SPICE frames

In [ ]:
# Here we define some general inputs needed by subsequent calculations

# Define the time span of the calculation
ref_epc = "2010 may 10 09:25:00"  # Start epoch of the computation
duration = 1000     # Time span (seconds)
timestep = 100      # Time step (seconds)

epc_et0 = sp.str2et(ref_epc) 
epc_et1 = epc_et0 + duration
epochs = epochRange2(startEpoch=epc_et0, endEpoch=epc_et1, step=timestep)  # This is a utility function to create equally spaed epoche

# Ray tracing options
spacing = 0.01      # Spacing between rays (meters)

# We use the 3D shape of the spacecraft, thus the 3D files must be provided
# Here we always use .obj fles, but other formats can be used
# (as long as they're readable by trimesh)
obj_path = '../example_data/LRO/'

# Some physical constants
base_flux = 1361.5   # Sun flux in W/m2
ref_radius = 1737.4  # Reference radius of the Moon for eclipse calculations
n_cores = 10         # number of cores for parallel calculations




### Spacecraft shape definition
Here we show how to define the spacecraft shape

In [ ]:
# The spacecraft mass can be a float, int or a xarray with times and values [kg]
# Here we use the simple case of a constant mass. See other examples for more details
# on using a time-variable mass
sc_mass = 2000  # kg


# 3D shape definition

# Define the Spacecraft Object 
# The spacecraft object is basically a dictionary
# containing each component of the spacecraft. 
# Each component must have a 3d shape,  a reference frame,
# a position and thermo-optical coeffiients (diffusive and specular)
# 
# The reference frame of each object can be defined as the name
# of a spice frame (frame_type = 'Spice') thus CK kernels must be provided
# or as a User Defined frame (frame_type = 'UD'). In this case the user must provide the 
# rotation matrix between the object frame and the base frame of the overall spaecraft.
# Here we use the Spice frames defined for LRO. In another example we'll see how to 
# use custom frames to achieve different results. 


#(Refer to the class documentation for further details)

lro = Spacecraft( name = 'LRO',
                 
				  base_frame = 'LRO_SC_BUS', 					     # Name of the spacecraft body-fixed frame
      
                  mass = sc_mass,
      
				  spacecraft_model = {						         # Define a spacecraft model
                          
					'LRO_BUS': { 
							 'file' : obj_path + 'bus_rotated.obj',	 # .obj file of the spacecraft component
							 'frame_type': 'Spice',				     # type of frame (can be 'Spice' or 'UD')
							 'frame_name': 'LRO_SC_BUS',			 # Name of the frame
							 'center': [0.0,0.0,0.0],			     # Origin of the component in the S/C frame
							 'diffuse': 0.1,				         # Diffuse reflect. coefficient
							 'specular': 0.3,				         # Specular reflect. coefficient
							 },

					'LRO_SA': {	
							'file': obj_path + 'SA_recentred.obj',
							'frame_type': 'Spice',
							'frame_name': 'LRO_SA',
							'center': [-1,-1.1, -0.1],
							'diffuse': 0,
							'specular': 0.3,
							},


					'LRO_HGA': { 	
							'file': obj_path + 'HGA_recentred.obj',
							'frame_type': 'Spice',
							'frame_name': 'LRO_HGA',
							'center':[-0.99,    -0.3,  -3.1],
							'diffuse': 0.2,
							'specular': 0.1,
							},
					}
				)


In [ ]:
# Let's visualize the spacecraft to make sure everything is set up properly
lro.info() # Print information about the spaceraft shape components
# Get the mesh of the whole spacecraft at a certain time
# The split option allows to retrieve the separate meshes for each component
mesh = lro.dump(epc_et0, split = True)
mesh_names = lro.spacecraft_model.keys()

# Use the visualization utility
plot_mesh(mesh, 
          title = 'LRO',
          labels = list(mesh_names),
          elev = 45, azim = 140 # These are the angles of the viewpoint
         )


To compute the SRP we need also to define 
1) The source body (Sun)
2) The solar pressure object




3) The eclipse body (the Moon in the case of LRO)
4) The ray tracer
5) The shdow model, which describes how the shadow is accounted for


In [ ]:
# In pyRTX for most uses, the Sun is modeled as a rectangular
# "pixel plane". This is substantially a source plane for all 
# the rays that will be tracked (traced). 

# Define the Sun rays object
rays = PixelPlane( spacecraft  = lro,         # Spacecraft object 
			       mode        = 'Dynamic',   # Mode: can be 'Dynamic' ( The sun orientation is computed from the kernels), or 'Fixed'
			       distance    = 100,	      # Distance of the ray origin from the spacecraft (in meters)
			       source      = 'Sun',       # Source body (used to compute the orientation of the rays wrt. spacecraft)
			       width       = 10,	      # Width of the pixel plane (in m)
			       height      = 10,          # Height of the pixel plane (in m)
			       ray_spacing = spacing,     # Ray spacing (in m)
				   )


# Define the ray tracer
rtx = RayTracer( lro,                    # Spacecraft object
                 rays,                   # pixelPlane object
                 kernel = 'Embree3',     # The RTX kernel to use (can be either Embree or Embree 3)
                 bounces = 1,            # The number of bounces to account for
                 diffusion = False,      # Account for secondary diffusion
                ) 



# Define the solar pressure object
srp = SolarPressure( lro, 
				     rtx,
				     baseflux       = base_flux,    # Here we use the None option to obtain the generalized geometry vector, used also for the computation of albedo and thermal infrared
				     shadowObj      = None,

				     )
# Compute the SRP acceleration 
accel = srp.compute(epochs, n_cores = 1) * 1e3

In [ ]:
# Define the Moon object
moon = Planet(  fromFile      = None,
                radius        = ref_radius,
                name          = 'Moon',
                bodyFrame     = 'MOON_PA',
                sunFixedFrame = 'GSE_MOON',
                units         = 'km',
                subdivs       = 5,
                )


# Define the Sun rays object
rays = PixelPlane( spacecraft  = lro,         # Spacecraft object 
			       mode        = 'Dynamic',   # Mode: can be 'Dynamic' ( The sun orientation is computed from the kernels), or 'Fixed'
			       distance    = 100,	      # Distance of the ray origin from the spacecraft
			       source      = 'Sun',       # Source body (used to compute the orientation of the rays wrt. spacecraft)
			       width       = 10,	      # Width of the pixel plane
			       height      = 10,          # Height of the pixel plane
			       ray_spacing = spacing,     # Ray spacing (in m)
				   )


# Define the ray tracer
rtx = RayTracer( lro,                    # Spacecraft object
                 rays,                   # pixelPlane object
                 kernel = 'Embree3',     # The RTX kernel to use
                 bounces = 2,            # The number of bounces to account for
                 diffusion = False,      # Account for secondary diffusion
                ) 

# Time initialization
tic = timeit.default_timer()
# Precomputation object. This object performs all the calls to spiceypy before 
# calculating the acceleration. This is necessary when calculating the acceleration
# with parallel cores.
prec = Precompute(epochs = epochs,)
prec.precomputeSolarPressure(lro, moon, correction='LT+S')
prec.dump()

# Define the shadow function object
shadow = SunShadow( spacecraft     = lro,
				    body           = 'Moon',
				    bodyShape      = moon,
				    limbDarkening  = 'Eddington',
        			precomputation = prec,
				    )

# Define the solar pressure object
srp = SolarPressure( lro, 
				     rtx,
				     baseflux       = base_flux,    # Here we use the None option to obtain the generalized geometry vector, used also for the computation of albedo and thermal infrared
				     shadowObj      = shadow,
					 precomputation = prec,
				     )

# Managing Error messages from trimesh
# (when concatenating textures, in this case, withouth .mtl definition, trimesh returns a warning that
#  would fill the stdout. Deactivate it for a clean output)
log = logging.getLogger('trimesh')
log.disabled = True

### ------------------------------------------------------------------------------------------------------- ###
### COMPUTATIONS

# Compute the SRP acceleration 
accel = srp.compute(epochs, n_cores = n_cores) * 1e3
        
log.disabled = False

# Always unload the SPICE kernels
sp.unload(METAKR)

### ... Elapsed time
toc = timeit.default_timer()
time_min = int(floor((toc-tic)/60))
time_sec = int(mod((toc-tic), 60))
print("")
print("\t Elapsed time: %d min, %d sec" %(time_min, time_sec))
print("")

### ------------------------------------------------------------------------------------------------------- ###
### PLOT

epochs  = [float( epc - epc_et0)/3600 for epc in epochs]

fig, ax = plt.subplots(3, 1, figsize=(14,8), sharex = True)

ax[0].plot(epochs, accel[:,0], linewidth = 2, color = "tab:blue")
ax[0].set_ylabel('X [m/s^2]')
ax[1].plot(epochs, accel[:,1], linewidth = 2, color = "tab:blue")
ax[1].set_ylabel('Y [m/s^2]')
ax[2].plot(epochs, accel[:,2], linewidth = 2, color = "tab:blue")
ax[2].set_ylabel('Z [m/s^2]')
ax[2].set_xlabel('Hours from t0')

plt.tight_layout()
plt.show()

### ------------------------------------------------------------------------------------------------------- ###